Import dependencies

In [3]:
from openai import OpenAI
from dotenv import load_dotenv
from extractor import fetch_website_links, fetch_website_contents
import os
import json

load_dotenv()

True

Define Open AI keys

In [ ]:
OPENROUTER_API_KEY= os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL= os.getenv("OPENROUTER_BASE_URL")


: 

Rail guards check

In [ ]:
if OPENROUTER_API_KEY:
    print("API_KEY found")
else:
    print("No API_KEY found")
    exit(1)

: 

Define OpenAI client library

In [ ]:
client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=OPENROUTER_API_KEY)

: 

Define system and user prompt 

In [ ]:

system_one_shot_prompt = """You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

: 

In [ ]:
def user_prompts(url):
    user_prompt = f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):

    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt
    

: 

Send request to open ai model

In [ ]:
def select_relevant_link(url):
  response = client.chat.completions.create(
          model="openai/gpt-3.5-turbo",
          messages=[
                {
                  "role": "system",
                  "content": system_one_shot_prompt
                },
                {
                    "role": "user",
                    "content": user_prompts(url)
                }
              ],
          response_format= {"type": "json_object"}

      )
  result = response.choices[0].message.content
  links = json.loads(result)

  print(f"Found {len(links['links'])} relevant links")
  return links


: 

In [ ]:
select_relevant_link("https://huggingface.co")

: 

Bronchure Generator


In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_link(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

: 